In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchtext.datasets import Multi30k
from torchtext.data import Field, BucketIterator
import numpy as np
import spacy
import random

import seaborn as sns
import matplotlib.pyplot as plt

from torch.nn import TransformerEncoder, TransformerEncoderLayer
from torch.nn import TransformerDecoder, TransformerDecoderLayer
import torch.nn.functional as F

import warnings
warnings.filterwarnings("ignore")

In [2]:
CFG = {"IN_LANG":"en", "OUT_LANG": "de"}

In [3]:
import spacy.cli
import en_core_web_sm
import de_core_news_sm


spacy.cli.download("en_core_web_sm")
spacy.cli.download("de_core_news_sm")


if CFG["IN_LANG"] == "en":
    spacy_in_lang = en_core_web_sm.load()
    spacy_out_lang = de_core_news_sm.load()
else:
    spacy_in_lang = de_core_news_sm.load()
    spacy_out_lang = en_core_web_sm.load()
    

✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
✔ Download and installation successful
You can now load the model via spacy.load('de_core_news_sm')


In [4]:
def tokenizer_in(text):
    return [tok.text for tok in spacy_in_lang.tokenizer(text)]

def tokenizer_out(text):
    return [tok.text for tok in spacy_out_lang.tokenizer(text)]

in_lang = Field(tokenize=tokenizer_in, lower=True)
out_lang = Field(tokenize=tokenizer_out, lower=True, init_token="<sos>", eos_token="<eos>")

In [5]:
train_data, valid_data, test_data = Multi30k.splits(
        exts=("."+CFG["IN_LANG"], "."+CFG["OUT_LANG"]), fields=(in_lang, out_lang ))

In [6]:
in_lang.build_vocab(train_data, max_size=10000, min_freq=2)
out_lang.build_vocab(train_data, max_size=10000, min_freq=2)

In [7]:
class PositionalEncoding(nn.Module): #Positional encoding from: https://nlp.seas.harvard.edu/2018/04/03/attention.html
    def __init__(self, d_model, dropout=0.1, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)
        self.scale = nn.Parameter(torch.ones(1))

        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(
            0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0).transpose(0, 1)
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + self.scale * self.pe[:x.size(0), :]
        return self.dropout(x)

In [8]:
class TransformerModel(nn.Module):
    
    def __init__(self, intoken, outtoken ,hidden, enc_layers=2, dec_layers=2, dropout=.1, nheads=2, ff_model=128):
        super(TransformerModel, self).__init__()
        
        self.encoder = nn.Embedding(intoken, hidden)
        self.pos_encoder = PositionalEncoding(hidden, dropout)

        self.decoder = nn.Embedding(outtoken, hidden) 
        self.pos_decoder = PositionalEncoding(hidden, dropout)
        
        
        encoder_layers = TransformerEncoderLayer(d_model=hidden, nhead = nheads, dim_feedforward = ff_model, dropout=dropout, activation='relu')
        self.transformer_encoder = TransformerEncoder(encoder_layers, enc_layers)

        encoder_layers = TransformerDecoderLayer(hidden, nheads, ff_model, dropout, activation='relu')
        self.transformer_decoder = TransformerDecoder(encoder_layers, dec_layers)        

        self.fc_out = nn.Linear(hidden, outtoken)

        self.src_mask = None
        self.trg_mask = None
        self.memory_mask = None

        
    def generate_square_subsequent_mask(self, sz, sz1=None):
        
        if sz1 == None:
            mask = torch.triu(torch.ones(sz, sz), 1)
        else:
            mask = torch.triu(torch.ones(sz, sz1), 1)
            
        return mask.masked_fill(mask==1, float('-inf'))

    def make_len_mask_enc(self, inp):
        return (inp == in_pad_idx).transpose(0, 1)   #(batch_size, output_seq_len)
    
    def make_len_mask_dec(self, inp):
        return (inp == out_pad_idx).transpose(0, 1) #(batch_size, input_seq_len)
    


    def forward(self, src, trg): #SRC: (seq_len, batch_size)

        if self.trg_mask is None or self.trg_mask.size(0) != len(trg):
            self.trg_mask = self.generate_square_subsequent_mask(len(trg)).to(trg.device)
            

        #Adding padding mask
        src_pad_mask = self.make_len_mask_enc(src)
        trg_pad_mask = self.make_len_mask_dec(trg)
             

        #Add embeddings Encoder
        src = self.encoder(src)  #Embedding, (seq_len, batch_size, d_model)
        src = self.pos_encoder(src)   #Pos embedding
        
        
        #Add embedding decoder
        trg = self.decoder(trg) #(seq_len, batch_size, d_model)
        trg = self.pos_decoder(trg)

        
        memory = self.transformer_encoder(src, None, src_pad_mask)
        output = self.transformer_decoder(tgt = trg, memory = memory, tgt_mask = self.trg_mask, memory_mask = None, 
                                          tgt_key_padding_mask = trg_pad_mask, memory_key_padding_mask = src_pad_mask)

        output = self.fc_out(output)

        return output

In [12]:
class NoamOpt:
    "Optim wrapper that implements rate."
    def __init__(self, model_size, factor, warmup, optimizer):
        self.optimizer = optimizer
        self._step = 0
        self.warmup = warmup
        self.factor = factor
        self.model_size = model_size
        self._rate = 0
        
    def step(self):
        "Update parameters and rate"
        self._step += 1
        rate = self.rate()
        for p in self.optimizer.param_groups:
            p['lr'] = rate
        self._rate = rate
        self.optimizer.step()
        
    def rate(self, step = None):
        "Implement `lrate` above"
        if step is None:
            step = self._step
        return self.factor * \
            (self.model_size ** (-0.5) *
            min(step ** (-0.5), step * self.warmup ** (-1.5)))

In [13]:
#Training Hyperparameters
num_epochs = 60
batch_size = 128

maxlen = 50

In [14]:
#Model Hyperparameter
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
input_size_encoder = len(in_lang.vocab)
input_size_decoder = len(out_lang.vocab)
output_size = len(out_lang.vocab)

d_model = 256

import math 

model = TransformerModel(input_size_encoder, input_size_decoder ,d_model, enc_layers=1, dec_layers=1, dropout=.1, nheads=1, ff_model=1028).to(device)

In [15]:
in_pad_idx = in_lang.vocab.stoi['<pad>']
out_pad_idx = out_lang.vocab.stoi['<pad>']


train_iterator, valid_iterator, test_iterator = BucketIterator.splits(
    (train_data, valid_data, test_data),
    batch_size=batch_size,
    sort_within_batch=True,
    sort_key=lambda x: len(x.src),
    device=device,
)

criterion = nn.CrossEntropyLoss(ignore_index=out_pad_idx)
optimizer = NoamOpt(d_model, 1, 4000 ,optim.Adam(model.parameters(), lr=0))

In [16]:
def translate_sentence_transformer(model, sentence, max_length=50):
    model.eval()
    tokens = [token.text.lower() for token in spacy_in_lang(sentence)]

    text_to_indices = [in_lang.vocab.stoi[token] for token in tokens]

    # Convert to Tensor
    sentence_tensor = torch.LongTensor(text_to_indices).unsqueeze(1).to(device)

    preds = [out_lang.vocab.stoi[out_lang.init_token]]

    with torch.no_grad():
        
        emb_src = model.encoder(sentence_tensor)
        emb_src = model.pos_encoder(emb_src)

        memory = model.transformer_encoder(emb_src)

        for i in range(50):

            trg = torch.Tensor(preds).long().unsqueeze(1).to(device)
            trg = model.decoder(trg)
            trg = model.pos_decoder(trg)

            out = model.transformer_decoder(tgt = trg, memory = memory)
            out = model.fc_out(out)
            
            

            new = out.squeeze(1)[-1].argmax().item()
            preds.append(new)
            if new == out_lang.vocab.stoi["<eos>"]:
                break

    
    return [out_lang.vocab.itos[i] for i in preds]

In [17]:
def get_out_encoder(src):
    
    model.eval()
    tokens = [token.text.lower() for token in spacy_in_lang(src)]

    text_to_indices = [in_lang.vocab.stoi[token] for token in tokens]

    # Convert to Tensor
    sentence_tensor = torch.LongTensor(text_to_indices).unsqueeze(1).to(device)    

    with torch.no_grad():
        
        emb_src = model.encoder(sentence_tensor)
        emb_src = model.pos_encoder(emb_src)

        memory = model.transformer_encoder(emb_src)

        return memory

In [19]:
def beam(phrase, k):
    
    model.eval()
    memory = get_out_encoder(phrase)

    sos = out_lang.vocab.stoi["<sos>"]
    tgt = [sos]

    with torch.no_grad():

        trg = torch.Tensor(tgt).long().unsqueeze(1).to(device)
        trg = model.decoder(trg)
        trg = model.pos_decoder(trg)

        out = model.transformer_decoder(tgt = trg, memory = memory)
        out = F.softmax(model.fc_out(out), dim=-1)[-1].squeeze()

        args = out.argsort()[-k:].detach().cpu().numpy()
        probs = out[args].detach().cpu().numpy()

        probs = np.log(probs)
        possible = list(zip([tgt + [args[i]] for i in range(k)], probs))
        
        for i in range(maxlen):

            test=  []
            for j in range(k):

                tmp_tgt, tmp_prob = possible[j]

                if tmp_tgt[-1] == out_lang.vocab.stoi["<eos>"]:
                    test.append(possible[j])

                else:
                    trg = torch.Tensor(tmp_tgt).long().unsqueeze(1).to(device)
                    trg = model.decoder(trg)
                    trg = model.pos_decoder(trg)

                    out = model.transformer_decoder(tgt = trg, memory = memory)
                    out = F.softmax(model.fc_out(out), dim=-1)[-1].squeeze()

                    tmp_args = out.argsort()[-k:].detach().cpu().numpy()
                    tmp_probs = out[tmp_args].detach().cpu().numpy()
                    tmp_probs = (tmp_prob + np.log(tmp_probs))/(len(tmp_tgt)-1)

                    for r in range(k): 
                        test.append((tmp_tgt + [tmp_args[r]], tmp_probs[r]))


            possible = sorted(test, key=lambda x:x[1], reverse=True)[:k]
            
    return possible

In [20]:
def convert(x):
    
    sentence = x[0]
    sentence = [out_lang.vocab.itos[i] for i in sentence]
    
    return (" ".join(sentence), x[1])

In [21]:
def run_epoch():
    
    model.train()
    
    total_loss = 0
    
    for batch_idx, batch in enumerate(train_iterator):
        
        inp_data = batch.src.to(device)
        target = batch.trg.to(device)

        output = model(inp_data, target[:-1, ])
        output = output.reshape(-1, output.shape[2])
        target = target[1:].reshape(-1)

        optimizer.optimizer.zero_grad()
        loss = criterion(output, target)
        total_loss += loss
        
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1)
        optimizer.step()
        
        
    return total_loss/len(train_iterator)

In [22]:
def run_validation():
    
    model.eval()
    
    total_loss = 0
    
    with torch.no_grad():
    
        for batch_idx, batch in enumerate(valid_iterator):

            inp_data = batch.src.to(device)
            target = batch.trg.to(device)

            output = model(inp_data, target[:-1, ])
            output = output.reshape(-1, output.shape[2])
            target = target[1:].reshape(-1)

            loss = criterion(output, target)
            total_loss += loss


    return total_loss/len(valid_iterator)

In [24]:
sentence = 'a man in green holds a guitar while the other man observes his shirt .'


best_loss = 6486468 

for epoch in range(num_epochs):
    
    print(f'Epoch [{epoch} / {num_epochs}]\n')
    
    loss = run_epoch()
    validation_loss = run_validation()

    translated_sentence = translate_sentence_transformer(model, sentence)
    out = beam(sentence, 3)   
    
    print(f"Translated example sentence: \n {list(map(convert, out[:2]))}")
    print(f"Greedy: {translated_sentence}")
    
    print(f"\n Train loss {loss} | Validation loss {validation_loss} \n \n")
    
    if validation_loss < best_loss:
        torch.save(model.state_dict(), "../models/new_transformer")
        best_loss = validation_loss
    

Epoch [0 / 60]

Translated example sentence: 
 [('<sos> ein mann in einem grünen gitarre und ein mann und ein mann in der hand . <eos>', -0.002370434), ('<sos> ein mann in grünen oberteil hält eine gitarre , während ein mann in der hand . <eos>', -0.0024878283)]
Greedy: ['<sos>', 'ein', 'mann', 'in', 'grünen', 'oberteil', 'hält', 'eine', 'gitarre', ',', 'während', 'ein', 'mann', 'in', 'der', 'hand', '.', '<eos>']

 Train loss 2.9315173625946045 | Validation loss 2.6100471019744873 
 

Epoch [1 / 60]

Translated example sentence: 
 [('<sos> ein mann in einem grünen gitarre , während ein mann in der nähe eines mannes . <eos>', -0.00122195), ('<sos> ein mann im grünen oberteil hält einen anderen mann , während ein mann in der hand . <eos>', -0.001357117)]
Greedy: ['<sos>', 'ein', 'mann', 'in', 'einem', 'grünen', 'gitarre', ',', 'während', 'andere', 'mann', 'in', 'der', 'nähe', 'eines', 'mannes', '.', '<eos>']

 Train loss 2.6698379516601562 | Validation loss 2.3837783336639404 
 

Epoch [